# `yacman` features and usage

This short tutorial show you the features of `yacman` package in action.

First, let's prepare some data to work with

In [1]:
import yaml
yaml_dict = {'cfg_version': 0.1, 'lvl1': {'lvl2': {'lvl3': {'entry': ['val1', 'val2']}}}}
yaml_str = """\
cfg_version: 0.1
lvl1:
  lvl2:
    lvl3:
      entry: ["val1","val2"]
"""
filepath = "test.yaml"

with open(filepath, 'w') as f:
    data = yaml.dump(yaml_dict, f)
    
import yacman

##  `YacAttMap` object creation

There are multiple ways to initialize an object of `YacAttMap` class:

1. **Read data from a YAML-formatted file**

In [2]:
yacmap = yacman.YacAttMap(filepath=filepath)
yacmap

cfg_version: 0.1
lvl1:
  lvl2:
    lvl3:
      entry: ['val1', 'val2']

2. **Read data from an `entries` mapping**

In [3]:
yacmap = yacman.YacAttMap(entries=yaml_dict)
yacmap

cfg_version: 0.1
lvl1:
  lvl2:
    lvl3:
      entry: ['val1', 'val2']

3. **Read data from a YAML-formatted string**

In [4]:
yacmap = yacman.YacAttMap(yamldata=yaml_str)
yacmap

cfg_version: 0.1
lvl1:
  lvl2:
    lvl3:
      entry: ['val1', 'val2']

## File locks; race-free writing
Instances of `YacAttMap` class support race-free writing and file locking, so that **it's safe to use them in multi-user contexts**

They can be created with or without write capabilities. Writable objects create a file lock, which prevents other processes managed by `yacman` from updating the source config file.

`writable` argument in the object constructor can be used to toggle writable mode. The source config file can be updated on disk (using `write` method) only if the `YacAttMap` instance is in writable mode

In [5]:
yacmap = yacman.YacAttMap(filepath=filepath, writable=False)


try:
    yacmap.write()
except OSError as e:
    print("Error caught: {}".format(e))

Error caught: You can't call write on an object that was created in read-only mode.


The write capabilities can be granted to an object:

In [6]:
yacmap = yacman.YacAttMap(filepath=filepath, writable=False)
yacmap.make_writable()
yacmap.write()

'/Users/mstolarczyk/Uczelnia/UVA/code/yacman/docs/test.yaml'

Or withheld:

In [7]:
yacmap.make_readonly()

True

If a file is currently locked by other `YacAttMap` object. The object will not be made writable/created with write capabilities until the lock is gone. If the lock persists, the action will fail (with a `RuntimeError`) after a selected `wait_time`, which is 10s by default:

In [8]:
yacmap = yacman.YacAttMap(filepath=filepath, writable=True)

try:
    yacmap1 = yacman.YacAttMap(filepath=filepath, writable=True, wait_max=1)
except RuntimeError as e:
    print("\nError caught: {}".format(e))
yacmap.make_readonly()

Waiting for file lock: lock.test.yaml ....
Error caught: The maximum wait time (1) has been reached and the lock file still exists.


True

Lastly, the `YacAttMap` class instances **can be used in a context manager**. This way the source config file will be locked, possibly updated (depending on what the user chooses to do), safely written to and unlocked with a single line of code:

In [9]:
yacmap = yacman.YacAttMap(filepath=filepath)

with yacmap as y:
    y.test = "test"

yacmap1 = yacman.YacAttMap(filepath=filepath)
yacmap1

cfg_version: 0.1
lvl1:
  lvl2:
    lvl3:
      entry: ['val1', 'val2']
test: test

## Key aliases in `AliasedYacAttMap`

`AliasedYacAttMap` is a child class of `YacAttMap` that supports top-level key aliases.

### Defining the aliases

There are two ways the aliases can be defined at the object construction stage:

1. By passing a literal aliases dictionary
2. By passing a function to be executed on the object itself that returns the dictionary

In any case, the resulting aliases mapping has to follow the format presented below:

In [13]:
aliases = {
    "key_1": ["first_key", "key_one"], 
    "key_2": ["second_key", "key_two", "fav_key"],
    "key_3": ["third_key", "key_three"]
}

#### Literal aliases dictionary
The `aliases` argument in the `AliasedYacAttmap` below is a Python `dict` with that maps the object keys to collection of aliases (Python `list`s of `str`). This format is strictly enforced.

In [14]:
aliased_yacmap = yacman.AliasedYacAttMap(entries={'key_1': 'val_1', 'key_2': 'val_2', 'key_3': 'val_3'},
                                         aliases=aliases)

Having set the aliases we can key the object either with the literal key or the aliases:

In [15]:
aliased_yacmap["key_1"] == aliased_yacmap["first_key"]

True

In [16]:
aliased_yacmap["key_two"] == aliased_yacmap["fav_key"]

True

#### Aliases returning function

The `aliases` argument in the `AliasedYacAttmap` below is a Python `callable` that takes the obejcect itself as an argument and returns the desired aliases mapping. This is especially useful when the object itself contains the aliases definition, for example:

In [22]:
entries={
    'key_1': {'value': 'val_1', 'aliases': ['first_key']},
    'key_2': {'value': 'val_2', 'aliases': ['second_key']},
    'key_3': {'value': 'val_3', 'aliases': ['third_key']}
}

In [23]:
aliased_yacmap = yacman.AliasedYacAttMap(entries=entries, aliases=lambda x: {k: v.__getitem__("aliases", expand=False) for k, v in x.items()})

In [24]:
aliased_yacmap

AliasedYacAttMap
key_1:
  value: val_1
  aliases: ['first_key']
key_2:
  value: val_2
  aliases: ['second_key']
key_3:
  value: val_3
  aliases: ['third_key']

In [ ]:
aliased_yacmap["key_1"] == 